In [1]:
import sys
import time
import json
from stable_baselines3 import PPO
from stable_baselines3.common.utils import set_random_seed

sys.path.append("../..")
from thesis.envs.minimatrix import MiniMatrix
from thesis.callbacks.minimatrix import MiniMatrixCallback
from thesis.policies.agv_routing_attention import AgvRoutingFE
import numpy as np
seed = 42
set_random_seed(seed)

In [3]:
import torch
import torch.nn as nn
m = nn.Softmax()
input = torch.randn(1,2)
output = m(input)

C:\Users\Wegma\AppData\Local\Temp\ipykernel_11280\3221464123.py:5: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  output = m(input)


In [4]:
output

tensor([[0.8917, 0.1083]])

In [2]:
models_dir = "../../models/MiniMatrix_Routing_Attn"
logdir = "../../logs/MiniMatrix_Routing_Attn"
fleetsize = 4
max_fleetsize = 10
run_name = f"PPO-{fleetsize}-{max_fleetsize}-{time.strftime('%d_%m-%H_%M_%S')}-{seed}"

In [3]:
env_args = dict(
        reward_target = 10, 
        reward_distance = 5,
        # reward_block = -5, 
        dispatchinginterval=120,
        routinginterval = 2,
        withCollisions = False,
        # blockTimeout = 5,
        # routingOnNode = False,
        # coordinates = False,
        includeNodesInReach = True,
    )

In [11]:
env = MiniMatrix(
    "../../envs/MiniMatrix.zip", 
    max_seconds = 2*60, 
    fleetsize = fleetsize, 
    max_fleetsize=max_fleetsize, 
    config_args = env_args,
    port=51149
)

c:\Users\Wegma\.conda\envs\thesis\lib\site-packages\alpyne\client\utils.py:124: UserWarning: Unzipping to temporary directory (C:\Users\Wegma\AppData\Local\Temp\alpyne_75566_fx_34gf_)
  warn(f"Unzipping to temporary directory ({tmp_dir})")


In [64]:
env.run.get_observation().rew

10.5

In [65]:
obs = env._convert_from_observation(env.run.get_observation()).reshape(10,10)
obs[:,:8]

tensor([[1.0000, 0.4773, 0.8837, 0.5455, 0.8837, 0.5455, 0.8837, 0.0000],
        [1.0000, 0.8182, 0.8837, 0.8182, 0.8837, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.6023, 0.8837, 0.5455, 0.8837, 0.4773, 1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.0000, 0.6023, 1.0000, 0.6023, 1.0000, 0.7909, 0.7209, 0.0000]])

In [66]:
dx = obs[:,1] - obs[:,5]
dy = obs[:,2] - obs[:,6]
def getAction(dx, dy):
    if abs(dx) > abs(dy):
        if dx > 0:
            return 1
        if dx < 0:
            return 3
    else:
        if dy > 0:
            return 2
        if dy < 0:
            return 4
    return 0
actions = [getAction(x,y) for x,y in zip(dx, dy)]
env.run.take_action(env._convert_to_action(actions))
actions

[3, 2, 0, 0, 0, 0, 0, 1, 0, 2]